# Trial 2
- Welcome to another trial of AI Systems Project: VisionARM
- Date: 26th October, 2025.

In [2]:
print("Welcome to another trial of AI Systems Project:VisionARM")

Welcome to another trial of AI Systems Project:VisionARM


In [3]:
import cv2
import mediapipe as mp
import numpy as np

In [4]:
def get_finger_states(hand_landmarks, hand_label):
    """
    Analyzes hand landmarks to determine the state (0 or 1) for each of the 5 fingers.
    Returns a list of 5 states [thumb, index, middle, ring, pinky].
    Returns [0, 0, 0, 0, 0] if hand is not detected.
    """
    states = []
    if hand_landmarks is None:
        return [0] * 5
        
    lm = hand_landmarks.landmark

    # Landmark indices for tips and PIP joints
    tip_ids = [4, 8, 12, 16, 20]
    pip_ids = [3, 6, 10, 14, 18] # Proximal Interphalangeal joints

    # 1. Thumb Logic
    try:
        if hand_label == 'Left':
            # Check if thumb tip is to the right of the thumb PIP joint
            if lm[tip_ids[0]].x > lm[pip_ids[0]].x:
                states.append(1) # Open
            else:
                states.append(0) # Closed
        elif hand_label == 'Right':
            # Check if thumb tip is to the left of the thumb PIP joint
            if lm[tip_ids[0]].x < lm[pip_ids[0]].x:
                states.append(1) # Open
            else:
                states.append(0) # Closed

        # 2. Four Fingers Logic
        for i in range(1, 5): # For index, middle, ring, pinky
            # Check if finger tip is above the finger PIP joint
            if lm[tip_ids[i]].y < lm[pip_ids[i]].y:
                states.append(1) # Open
            else:
                states.append(0) # Closed
    
    except Exception as e:
        print(f"Error in finger state detection: {e}")
        return [0] * 5
            
    return states

print("Helper function 'get_finger_states' defined.")

Helper function 'get_finger_states' defined.


In [5]:
def main():
    # --- 1. Initialization ---
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    # Initialize MediaPipe Hands
    # We set max_num_hands=2 to detect both hands
    hands = mp_hands.Hands(
        max_num_hands=2,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5
    )

    # Initialize OpenCV Video Capture
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open video stream from webcam.")
        return
    
    print("Starting webcam feed. Press 'ESC' to quit.")

    # Variable to store the *latest* finger data
    # This will be overwritten each frame.
    latest_finger_data = {}

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # --- 2. Frame Processing ---
        
        # Flip the image horizontally for a selfie-view display
        # Also convert the BGR image to RGB
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Get image dimensions
        h, w, _ = image.shape

        # --- 3. Data Extraction and Encoding ---

        # Define the 10 features with 0 as default
        finger_data = {
            'left_thumb': 0, 'left_index': 0, 'left_middle': 0, 'left_ring': 0, 'left_pinky': 0,
            'right_thumb': 0, 'right_index': 0, 'right_middle': 0, 'right_ring': 0, 'right_pinky': 0
        }
        feature_names = ["thumb", "index", "middle", "ring", "pinky"]

        if results.multi_hand_landmarks:
            for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                # Get hand label (Left or Right)
                hand_label = results.multi_handedness[hand_idx].classification[0].label
                
                # Get finger states (0 or 1)
                states = get_finger_states(hand_landmarks, hand_label)
                
                # Update the dictionary
                prefix = 'left_' if hand_label == 'Left' else 'right_'
                for i, name in enumerate(feature_names):
                    finger_data[prefix + name] = states[i]
                
                # Draw landmarks on the image
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

        # --- 4. Store Latest Frame Data ---
        # Overwrite the 'latest_finger_data' variable with the data from this frame
        latest_finger_data = finger_data.copy()

        # Optional: Print the latest data to the console in real-time
        # Use carriage return '\r' to print on the same line
        print(f"Latest Data: {latest_finger_data}    ", end="\r")


        # --- 5. Display Encoded Data on Frame ---
        
        # Display Left Hand Data
        y_pos = 30
        for key, value in finger_data.items():
            if key.startswith('left'):
                cv2.putText(image, f"{key}: {value}", (10, y_pos), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                y_pos += 30

        # Display Right Hand Data
        y_pos = 30
        for key, value in finger_data.items():
            if key.startswith('right'):
                cv2.putText(image, f"{key}: {value}", (w - 250, y_pos), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                y_pos += 30

        # --- 6. Show Image and Handle Exit ---
        
        cv2.imshow('MediaPipe Hands', image)

        # Exit loop when 'ESC' is pressed
        if cv2.waitKey(5) & 0xFF == 27:
            break

    # --- 7. Cleanup and Data Display ---
    cap.release()
    cv2.destroyAllWindows()
    print("\nWebcam feed stopped.") # Add newline to clear the real-time print
    
    # Print the very last recorded data
    print(f"\nFinal recorded data: {latest_finger_data}")

In [6]:
# --- 1. Initialization ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [7]:
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)

I0000 00:00:1761830346.057424   13133 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1761830346.070291   14093 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761830346.074807   14093 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [8]:
# Initialize OpenCV Video Capture
cap = cv2.VideoCapture(0)

# Variable to store the *latest* finger data
latest_finger_data = {}

In [1]:
!pip install mediapipe opencv-python scikit-learn

import mediapipe as mp
import cv2
import numpy as np
from sklearn.svm import SVC
import joblib

In [2]:
svm_model = joblib.load('./SVM_Models/best_label_classification_model.pkl')
svm_model

,steps,"[('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False


In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.5)

# Helper: Decides if a finger is open/closed using landmark z-coordinates (folding direction).
def encode_fingers(hand_landmarks):
    # Returns list of 5 binary features: [thumb, index, middle, ring, pinky]
    finger_status = []
    tips = [mp_hands.HandLandmark.THUMB_TIP,
            mp_hands.HandLandmark.INDEX_FINGER_TIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
            mp_hands.HandLandmark.RING_FINGER_TIP,
            mp_hands.HandLandmark.PINKY_TIP]
    dip = [mp_hands.HandLandmark.THUMB_IP,
           mp_hands.HandLandmark.INDEX_FINGER_DIP,
           mp_hands.HandLandmark.MIDDLE_FINGER_DIP,
           mp_hands.HandLandmark.RING_FINGER_DIP,
           mp_hands.HandLandmark.PINKY_DIP]
    for t, d in zip(tips, dip):
        try:
            tip = hand_landmarks.landmark[t]
            dip_ = hand_landmarks.landmark[d]
            is_open = 1 if tip.y < dip_.y else 0  # simple heuristic: tip above dip = open
        except Exception:
            is_open = 0  # missing value
        finger_status.append(is_open)
    return finger_status

I0000 00:00:1762008796.677637   31394 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1762008796.683136   94248 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762008796.687933   94246 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
buffer = None  # stores the latest 10-value feature list

def process_frame(frame):
    global buffer
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    # Default feature: all 0 (length 10)
    features = [0]*10
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            hand_type = results.multi_handedness[hand_idx].classification[0].label
            fingers = encode_fingers(hand_landmarks)
            if hand_type == "Left":
                features[:5] = fingers
            elif hand_type == "Right":
                features[5:] = fingers
    buffer = features  # replace previous stored value

In [ ]:
 # Placeholder; retrain/fit on real collected features!

def run_svm_on_buffer():
    global buffer
    print("SVM input:", buffer)
    prediction = svm_model.predict([buffer])  # model prediction
    print("SVM prediction:", prediction)
    # After prediction, buffer will be overwritten in next frame

In [10]:
cap = cv2.VideoCapture(0)
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        process_frame(frame)
        run_svm_on_buffer()
        cv2.imshow("MediaPipe Hands", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']


/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']


/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']


...

/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: 

SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']


/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: 

SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']
SVM input: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SVM prediction: ['stop']


/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/Users/shivanshu_ade/Documents/anaconda3/envs/mp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
